In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')

In [3]:
import numpy as np

def squarebounds(_open=None, close=None, alpha=0):

    N = len(close)

    up1, up2, dn1, dn2, bull, bear = np.zeros((6, N))

    up1[0] = dn1[0] = close[0]
    up2[0] = dn2[0] = close[0] ** 2
       
    for i in range(1, N):
        
        up1[i] = max(close[i], _open[i], up1[i - 1] - (up1[i - 1] - close[i]) / alpha)
        dn1[i] = min(close[i], _open[i], dn1[i - 1] + (close[i] - dn1[i - 1]) / alpha)

        up2[i] = max(close[i] ** 2, _open[i] ** 2, up2[i - 1] - (up2[i - 1] - close[i] ** 2) / alpha)
        dn2[i] = min(close[i] ** 2, _open[i] ** 2, dn2[i - 1] + (close[i] ** 2 - dn2[i - 1]) / alpha)
             
        
        bull[i] = np.sqrt(up2[i] - up1[i] ** 2)
        bear[i] = np.sqrt(dn2[i] - dn1[i] ** 2)
        
    bear *= -1
    
    bull_d1 = np.diff(bull)
    bull_d2 = np.diff(bull_d1)

    bear_d1 = np.diff(bear)
    bear_d2 = np.diff(bear_d1)
    
    long_condition = bull_d1[-1] < 0 and bear[i] == 0
    short_condition = bear_d1[-1] > 0
    
    return long_condition, short_condition

In [4]:
import priceanalytics.data as data

big_df = data.alpaca_download_single(
    'AAPL',     
    past_days=30, 
    interval=5  
)
dfs = data.split_alpaca_on_day(big_df)

/Users/justus/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
import priceanalytics.backtest as bt
import priceanalytics.plot as plot
import matplotlib.pyplot as plt
    
def test_strategy(df, alpha=10):
    
    with bt.Backtester(df) as b:
     
        for i, fr in b:
            
            if i < 2: continue
        
                      
            long, short = squarebounds(
                _open=fr['open'].to_numpy(),
                close=fr['close'].to_numpy(),
                alpha=alpha
            )
            
            if long:
                if b.buy():
                    print('bought', i, b.price())

            
            elif short:
                if b.sell():
                    print('sold', i, b.price())
            
        return b.results(1)

r = test_strategy(dfs[-1])
print(r)

bought 21 194.205
sold 28 194.215
bought 30 194.23
sold 46 194.1801
-0.00020543315734078416
